In [13]:
import numpy as np 
import pandas as pd 
import bisect

# plot
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML
import matplotlib.image as image
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
import matplotlib.image as mpimg
import flag
import plotly.express as px
import plotly.graph_objects as go
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot
init_notebook_mode(connected=True)

# files
import urllib.request
import cairosvg

import seaborn as sns
from IPython.core.display import HTML

pd.set_option('display.max_colwidth', -1)

<ipython-input-13-e1d37169ecc5>:28: FutureWarning:

Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.



In [8]:
import pandas as pd 
import numpy as np 
  
# Seeding random data from numpy 
np.random.seed(24) 
  
# Making the DatFrame 
df = pd.DataFrame({'A': np.linspace(1, 10, 10)}) 
df = pd.concat([df, pd.DataFrame(np.random.randn(10, 4),  
                                 columns=list('BCDE'))], axis=1) 
  
# DataFrame without any styling 
print("Original DataFrame:\n") 
print(df) 
print("\nModified Stlying DataFrame:") 
df.style.set_properties(**{'background-color': 'black', 
                           'color': 'green'})

Original DataFrame:

      A         B         C         D         E
0  1.0   1.329212 -0.770033 -0.316280 -0.990810
1  2.0  -1.070816 -1.438713  0.564417  0.295722
2  3.0  -1.626404  0.219565  0.678805  1.889273
3  4.0   0.961538  0.104011 -0.481165  0.850229
4  5.0   1.453425  1.057737  0.165562  0.515018
5  6.0  -1.336936  0.562861  1.392855 -0.063328
6  7.0   0.121668  1.207603 -0.002040  1.627796
7  8.0   0.354493  1.037528 -0.385684  0.519818
8  9.0   1.686583 -1.325963  1.428984 -2.089354
9  10.0 -0.129820  0.631523 -0.586538  0.290720

Modified Stlying DataFrame:


,A,B,C,D,E
0,1.000000,1.329212,-0.770033,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,-0.481165,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


In [14]:
covid_data = pd.read_csv("../data/covid-data .csv")
covid_data

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
0,ABW,North America,Aruba,2020-03-13,2.0,2.0,NaN,0.0,0.0,NaN,...,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29
1,ABW,North America,Aruba,2020-03-19,NaN,NaN,0.286,NaN,NaN,0.0,...,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29
2,ABW,North America,Aruba,2020-03-20,4.0,2.0,0.286,0.0,0.0,0.0,...,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29
3,ABW,North America,Aruba,2020-03-21,NaN,NaN,0.286,NaN,NaN,0.0,...,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29
4,ABW,North America,Aruba,2020-03-22,NaN,NaN,0.286,NaN,NaN,0.0,...,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41587,NaN,NaN,International,2020-08-31,696.0,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41588,NaN,NaN,International,2020-09-01,696.0,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41589,NaN,NaN,International,2020-09-02,696.0,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41590,NaN,NaN,International,2020-09-03,696.0,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
covid_df = covid_data[['iso_code','date', 'location', 'continent', 'total_deaths', 'total_cases', 'population']]

In [16]:
covid_df = covid_df[covid_df.location != 'World']
covid_df = covid_df[covid_df.location != 'International']
covid_df.head()

,iso_code,date,location,continent,total_deaths,total_cases,population
0,ABW,2020-03-13,Aruba,North America,0.0,2.0,106766.0
1,ABW,2020-03-19,Aruba,North America,NaN,NaN,106766.0
2,ABW,2020-03-20,Aruba,North America,0.0,4.0,106766.0
3,ABW,2020-03-21,Aruba,North America,NaN,NaN,106766.0
4,ABW,2020-03-22,Aruba,North America,NaN,NaN,106766.0


In [17]:
covid_last_df = covid_df[covid_df.date == '2020-09-04']
covid_last_df.head()

,iso_code,date,location,continent,total_deaths,total_cases,population
170,ABW,2020-09-04,Aruba,North America,13.0,2292.0,106766.0
419,AFG,2020-09-04,Afghanistan,Asia,1410.0,38288.0,38928341.0
586,AGO,2020-09-04,Angola,Africa,112.0,2777.0,32866268.0
748,AIA,2020-09-04,Anguilla,North America,0.0,3.0,15002.0
928,ALB,2020-09-04,Albania,Europe,301.0,9844.0,2877800.0


In [18]:
covid_last_df = covid_last_df.drop(['date'], axis=1)
covid_last_df.head()

,iso_code,location,continent,total_deaths,total_cases,population
170,ABW,Aruba,North America,13.0,2292.0,106766.0
419,AFG,Afghanistan,Asia,1410.0,38288.0,38928341.0
586,AGO,Angola,Africa,112.0,2777.0,32866268.0
748,AIA,Anguilla,North America,0.0,3.0,15002.0
928,ALB,Albania,Europe,301.0,9844.0,2877800.0


In [19]:
flags = pd.read_csv("../data/Country_Flags.csv")
flags.head()

,Country,Images File Name,ImageURL
0,Afghanistan,Flag_of_Afghanistan.svg,https://upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Afghanistan.svg
1,Albania,Flag_of_Albania.svg,https://upload.wikimedia.org/wikipedia/commons/3/36/Flag_of_Albania.svg
2,Algeria,Flag_of_Algeria.svg,https://upload.wikimedia.org/wikipedia/commons/7/77/Flag_of_Algeria.svg
3,Andorra,Flag_of_Andorra.svg,https://upload.wikimedia.org/wikipedia/commons/1/19/Flag_of_Andorra.svg
4,Angola,Flag_of_Angola.svg,https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Angola.svg


In [20]:
# Rename Country to location to match the first dataframe
flags_df = flags.rename(columns ={'Country':'location'})
# Drop files name
flags_df = flags_df[['location', 'ImageURL']]

flags_df.head()

,location,ImageURL
0,Afghanistan,https://upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Afghanistan.svg
1,Albania,https://upload.wikimedia.org/wikipedia/commons/3/36/Flag_of_Albania.svg
2,Algeria,https://upload.wikimedia.org/wikipedia/commons/7/77/Flag_of_Algeria.svg
3,Andorra,https://upload.wikimedia.org/wikipedia/commons/1/19/Flag_of_Andorra.svg
4,Angola,https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Angola.svg


In [21]:
flags_df.ImageURL.replace(
    {'https://upload.wikimedia.org/wikipedia/commons/1/19/Flag_of_Honduras.svg':'',
     'https://upload.wikimedia.org/wikipedia/commons/1/19/Flag_of_Swaziland.svg': ''})

0      https://upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Afghanistan.svg                         
1      https://upload.wikimedia.org/wikipedia/commons/3/36/Flag_of_Albania.svg                             
2      https://upload.wikimedia.org/wikipedia/commons/7/77/Flag_of_Algeria.svg                             
3      https://upload.wikimedia.org/wikipedia/commons/1/19/Flag_of_Andorra.svg                             
4      https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Angola.svg                              
                                        ...                                                                
201    https://upload.wikimedia.org/wikipedia/commons/2/26/Flag_of_the_Sahrawi_Arab_Democratic_Republic.svg
202    https://upload.wikimedia.org/wikipedia/commons/4/4d/Flag_of_Somaliland.svg                          
203    https://upload.wikimedia.org/wikipedia/commons/1/12/Flag_of_South_Ossetia.svg                       
204    https://upload.wikime

In [22]:
democracy = pd.read_csv("../data/political_regime.csv")
democracy.head()

,Entity,Code,Year,Political Regime (OWID based on Polity IV and Wimmer & Min) (Score)
0,Afghanistan,AFG,1816,-6
1,Afghanistan,AFG,1817,-6
2,Afghanistan,AFG,1818,-6
3,Afghanistan,AFG,1819,-6
4,Afghanistan,AFG,1820,-6


In [23]:
democracy_df = democracy[democracy.Year==2015]
democracy_df = democracy_df.rename(
    columns =
    {
    'Entity':'location',
    'Political Regime (OWID based on Polity IV and Wimmer & Min) (Score)': 'democracy_score'})

democracy_df = democracy_df.drop(['Year', 'Code'], axis=1)
democracy_df.head()

,location,democracy_score
176,Afghanistan,-1
375,Albania,9
575,Algeria,2
775,Angola,-2
966,Argentina,9


In [24]:
democracy_df['score'] = [bisect.bisect_left([-6, -1,5,10], p)
                            for p in democracy_df.democracy_score]

In [25]:
def mask_country(country, score):
    mask = (democracy_df['location'] == country)
    democracy_df['score'][mask] = score

In [26]:
mask_country('Lebanon', 2)
mask_country('Madagascar', 2)
mask_country('Niger', 2)
mask_country('Zambia', 2)
mask_country('Lybia', 0)
mask_country('Chad', 0)
mask_country('Rwanda', 0)
mask_country('Yemen', 0)
mask_country('Nicaragua', 2)
mask_country('Venezuela', 0)
mask_country('Thailand', 0)
mask_country('Tajikistan', 0)
mask_country('Cambodia', 0)
mask_country('Singapore', 2)

<ipython-input-25-e2bd9e4288d0>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [27]:
democracy_df = democracy_df.drop(['democracy_score'], axis=1)

In [39]:
covid_democracy = pd.merge(covid_last_df, democracy_df, on='location')
covid_democracy = pd.merge(covid_democracy, flags_df, on='location')
#covid_democracy = pd.merge(covid_democracy, long_lat_df, on='location')
covid_democracy.head()

,iso_code,location,continent,total_deaths,total_cases,population,score,ImageURL
0,AFG,Afghanistan,Asia,1410.0,38288.0,38928341.0,1,https://upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Afghanistan.svg
1,AGO,Angola,Africa,112.0,2777.0,32866268.0,1,https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Angola.svg
2,ALB,Albania,Europe,301.0,9844.0,2877800.0,3,https://upload.wikimedia.org/wikipedia/commons/3/36/Flag_of_Albania.svg
3,ARE,United Arab Emirates,Asia,387.0,72154.0,9890400.0,0,https://upload.wikimedia.org/wikipedia/commons/c/cb/Flag_of_the_United_Arab_Emirates.svg
4,ARG,Argentina,South America,9155.0,439159.0,45195777.0,3,https://upload.wikimedia.org/wikipedia/commons/1/1a/Flag_of_Argentina.svg


In [40]:
covid_democracy['Regimes'] = covid_democracy['score'].replace([0, 1, 2, 3],
                                                            ['Autocracy', 'Closed Anocracy', 
                                                             'Open Anocracy','Democracy'])

In [41]:
covid_democracy.to_csv('../data/covid_democracy.csv', index=False)

In [42]:
covid_table = covid_democracy.drop(['iso_code'], axis=1)
covid_table = covid_table[['score','ImageURL','location', 'continent','total_deaths', 'total_cases', 'Regimes']]
covid_table.head()

,score,ImageURL,location,continent,total_deaths,total_cases,Regimes
0,1,https://upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Afghanistan.svg,Afghanistan,Asia,1410.0,38288.0,Closed Anocracy
1,1,https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Angola.svg,Angola,Africa,112.0,2777.0,Closed Anocracy
2,3,https://upload.wikimedia.org/wikipedia/commons/3/36/Flag_of_Albania.svg,Albania,Europe,301.0,9844.0,Democracy
3,0,https://upload.wikimedia.org/wikipedia/commons/c/cb/Flag_of_the_United_Arab_Emirates.svg,United Arab Emirates,Asia,387.0,72154.0,Autocracy
4,3,https://upload.wikimedia.org/wikipedia/commons/1/1a/Flag_of_Argentina.svg,Argentina,South America,9155.0,439159.0,Democracy


In [43]:
covid_table = covid_table.rename(columns={
    'location':'Country',
    'continent': 'Continent',
    'total_deaths': 'Total deaths',
    'total_cases': 'Total cases',
    'Regimes': 'Regimes',
    'ImageURL': 'Flags',
})

In [44]:
covid_table = covid_table.drop(['score'], axis=1)
covid_table

,Flags,Country,Continent,Total deaths,Total cases,Regimes
0,https://upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Afghanistan.svg,Afghanistan,Asia,1410.0,38288.0,Closed Anocracy
1,https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Angola.svg,Angola,Africa,112.0,2777.0,Closed Anocracy
2,https://upload.wikimedia.org/wikipedia/commons/3/36/Flag_of_Albania.svg,Albania,Europe,301.0,9844.0,Democracy
3,https://upload.wikimedia.org/wikipedia/commons/c/cb/Flag_of_the_United_Arab_Emirates.svg,United Arab Emirates,Asia,387.0,72154.0,Autocracy
4,https://upload.wikimedia.org/wikipedia/commons/1/1a/Flag_of_Argentina.svg,Argentina,South America,9155.0,439159.0,Democracy
...,...,...,...,...,...,...
148,https://upload.wikimedia.org/wikipedia/commons/2/21/Flag_of_Vietnam.svg,Vietnam,Asia,35.0,1046.0,Autocracy
149,https://upload.wikimedia.org/wikipedia/commons/8/89/Flag_of_Yemen.svg,Yemen,Asia,572.0,1980.0,Autocracy
150,https://upload.wikimedia.org/wikipedia/commons/a/af/Flag_of_South_Africa.svg,South Africa,Africa,14563.0,633015.0,Democracy
151,https://upload.wikimedia.org/wikipedia/commons/0/06/Flag_of_Zambia.svg,Zambia,Africa,292.0,12523.0,Open Anocracy


In [45]:
covid_table.style.hide_index()


Flags,Country,Continent,Total deaths,Total cases,Regimes
https://upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Afghanistan.svg,Afghanistan,Asia,1410.000000,38288.000000,Closed Anocracy
https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Angola.svg,Angola,Africa,112.000000,2777.000000,Closed Anocracy
https://upload.wikimedia.org/wikipedia/commons/3/36/Flag_of_Albania.svg,Albania,Europe,301.000000,9844.000000,Democracy
https://upload.wikimedia.org/wikipedia/commons/c/cb/Flag_of_the_United_Arab_Emirates.svg,United Arab Emirates,Asia,387.000000,72154.000000,Autocracy
https://upload.wikimedia.org/wikipedia/commons/1/1a/Flag_of_Argentina.svg,Argentina,South America,9155.000000,439159.000000,Democracy
https://upload.wikimedia.org/wikipedia/commons/2/2f/Flag_of_Armenia.svg,Armenia,Asia,887.000000,44271.000000,Open Anocracy
https://upload.wikimedia.org/wikipedia/commons/8/88/Flag_of_Australia_%28converted%29.svg,Australia,Oceania,678.000000,26049.000000,Democracy
https://upload.wikimedia.org/wikipedia/commons/4/41/Flag_of_Austria.svg,Austria,Europe,735.000000,28495.000000,Democracy
https://upload.wikimedia.org/wikipedia/commons/d/dd/Flag_of_Azerbaijan.svg,Azerbaijan,Asia,541.000000,36899.000000,Autocracy
https://upload.wikimedia.org/wikipedia/commons/5/50/Flag_of_Burundi.svg,Burundi,Africa,1.000000,448.000000,Closed Anocracy


In [38]:
covid_table

,Country,Continent,Total deaths,Total cases,Regimes
Flags,,,,,
https://upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Afghanistan.svg,Afghanistan,Asia,1410.0,38288.0,Closed Anocracy
https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Angola.svg,Angola,Africa,112.0,2777.0,Closed Anocracy
https://upload.wikimedia.org/wikipedia/commons/3/36/Flag_of_Albania.svg,Albania,Europe,301.0,9844.0,Democracy
https://upload.wikimedia.org/wikipedia/commons/c/cb/Flag_of_the_United_Arab_Emirates.svg,United Arab Emirates,Asia,387.0,72154.0,Autocracy
https://upload.wikimedia.org/wikipedia/commons/1/1a/Flag_of_Argentina.svg,Argentina,South America,9155.0,439159.0,Democracy
...,...,...,...,...,...
https://upload.wikimedia.org/wikipedia/commons/2/21/Flag_of_Vietnam.svg,Vietnam,Asia,35.0,1046.0,Autocracy
https://upload.wikimedia.org/wikipedia/commons/8/89/Flag_of_Yemen.svg,Yemen,Asia,572.0,1980.0,Autocracy
https://upload.wikimedia.org/wikipedia/commons/a/af/Flag_of_South_Africa.svg,South Africa,Africa,14563.0,633015.0,Democracy


In [54]:
covid_table.style.set_properties(**{'background-color': 'black', 
                           'color': 'white' }).set_table_styles(
   [{
       'selector': 'th',
       'props': [('background-color', 'black'), ('color','white')]
   }]).hide_index()


Flags,Country,Continent,Total deaths,Total cases,Regimes
https://upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Afghanistan.svg,Afghanistan,Asia,1410.000000,38288.000000,Closed Anocracy
https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Angola.svg,Angola,Africa,112.000000,2777.000000,Closed Anocracy
https://upload.wikimedia.org/wikipedia/commons/3/36/Flag_of_Albania.svg,Albania,Europe,301.000000,9844.000000,Democracy
https://upload.wikimedia.org/wikipedia/commons/c/cb/Flag_of_the_United_Arab_Emirates.svg,United Arab Emirates,Asia,387.000000,72154.000000,Autocracy
https://upload.wikimedia.org/wikipedia/commons/1/1a/Flag_of_Argentina.svg,Argentina,South America,9155.000000,439159.000000,Democracy
https://upload.wikimedia.org/wikipedia/commons/2/2f/Flag_of_Armenia.svg,Armenia,Asia,887.000000,44271.000000,Open Anocracy
https://upload.wikimedia.org/wikipedia/commons/8/88/Flag_of_Australia_%28converted%29.svg,Australia,Oceania,678.000000,26049.000000,Democracy
https://upload.wikimedia.org/wikipedia/commons/4/41/Flag_of_Austria.svg,Austria,Europe,735.000000,28495.000000,Democracy
https://upload.wikimedia.org/wikipedia/commons/d/dd/Flag_of_Azerbaijan.svg,Azerbaijan,Asia,541.000000,36899.000000,Autocracy
https://upload.wikimedia.org/wikipedia/commons/5/50/Flag_of_Burundi.svg,Burundi,Africa,1.000000,448.000000,Closed Anocracy


In [56]:
covid = HTML(covid_table.to_html(escape=False,formatters=dict(Flags=path_to_image_html, score= row_class)))
covid

NameError: name 'path_to_image_html' is not defined